In [4]:
import pandas as pd
import numpy as np
import pickle
import bs4 
from keras.utils import to_categorical
from keras import utils
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences 
from keras.preprocessing import text, sequence
from keras.layers import Dense, Dropout, Activation, Embedding, Conv1D, MaxPool1D, Flatten, GlobalMaxPooling1D, BatchNormalization
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_20newsgroups
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from keras.layers import LSTM
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [5]:
#Bringing the datasets
#train=pd.read_csv('Devex_train.csv',encoding='latin-1')
import os
os.environ['OMP_NUM_THREADS'] = '4'


EMBEDDING_FILE = '/data/fasttext/crawl-300d-2M.vec'
test=pd.read_csv('Devex_test_questions.csv',encoding='latin-1')
sample_sub=pd.read_csv('Devex_submission_format.csv')    #,encoding='latin-1')
#submitted=pd.read_csv('submission_sdg1.csv')
#train = pd.read_csv('../input/train.csv').fillna(' ')
#test = pd.read_csv('../input/test.csv').fillna(' ')

#train = train['Text']
#test = test['Text']

In [ ]:
train['Text'] = train['Text'].apply(lambda x: bs4.BeautifulSoup(x, 'lxml').get_text())
test['Text'] = test['Text'].apply(lambda x: bs4.BeautifulSoup(x, 'lxml').get_text())
#train.head()

In [8]:
class_names =list(sample_sub.columns.values)[1:]
#train = train.reindex(columns=train.columns.tolist() + target_variables)
#filling the NaN values
#train=train.select_dtypes(include=['object','float']).fillna('None')

In [17]:
#Transforming the target variables into a useful format
train["targets"]=train["Label 1"].apply(lambda x: x[:5] )
targets=pd.get_dummies(train["targets"])
#targets.select_dtypes(include=['object']).fillna(0)
for column in train.columns[4:15]:
    train["targets"]=train[column].apply(lambda x: x[:5] if (type(x)==str) else "None")
    new_targets=pd.get_dummies(train["targets"])
    diff=list(set(list(targets.columns))-set(list(new_targets.columns)))
    new_targets[diff]=targets[diff]*0
    #print(len(new_targets.columns))
    targets=targets + new_targets
    #print(targets["3.b.2"][0])
    if column=="Label 12":
        break
#targets

In [ ]:
train_text = train['Text']
test_text = test['Text']

X_train = train["Text"].fillna("fillna").values
y_train = targets[list(submission.columns.values)[1:]].values
X_test = test["Text"].fillna("fillna").values


max_features = 100000
maxlen = 400
embed_size = 300

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train_tok = tokenizer.texts_to_sequences(X_train)
X_test_tok = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train_tok, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test_tok, maxlen=maxlen)
# Now split the data into train, validation and test. I randomly did this split
# so feel free to change the ratio for the splits.
X_train, X_test, Y_train, Y_test = train_test_split(x_train, Y, test_size=0.20, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.20, random_state=42)

In [18]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [19]:
#counting the number of words in each comment and attaching it as a column to the Dataframe
train['Word Count'] =  train['Text'].apply(lambda x: len(x.split (' ')))
train.sort_values(by=['Word Count'], ascending=False)

,Type,Text,Label 1,Label 2,Label 3,Label 4,Label 5,Label 6,Label 7,Label 8,...,3.9.2,3.9.3,3.a.1,3.b.1,3.b.2,3.b.3,3.c.1,3.d.1,targets,Word Count
2637,Open Opp,International Collaborative Research on Drug A...,3.b.2 - Total net official development assista...,3.5.1 - Coverage of treatment interventions (p...,"3.5.2 - Harmful use of alcohol, defined accord...",3.a.1 - Age-standardized prevalence of current...,3.4.2 - Suicide mortality rate,None,None,None,...,None,None,None,None,None,None,None,None,None,4311
1396,Open Opp,Advancing the Science of Geriatric Palliative ...,3.4.1 - Mortality rate attributed to cardiovas...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,4237
836,Grant,Alzheimer's Disease Research Centers ; ; ; ...,3.b.2 - Total net official development assista...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,4126
1348,Open Opp,Complex Technologies and Therapeutics Developm...,3.b.2 - Total net official development assista...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,4044
190,Open Opp,International Research Collaboration on Drug A...,3.b.2 - Total net official development assista...,3.5.1 - Coverage of treatment interventions (p...,"3.5.2 - Harmful use of alcohol, defined accord...",3.4.2 - Suicide mortality rate,3.a.1 - Age-standardized prevalence of current...,None,None,None,...,None,None,None,None,None,None,None,None,None,4015
1237,Grant,Model Organisms Screening Center for the Undia...,3.b.2 - Total net official development assista...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,3941
2189,Open Opp,Advancing Translational and Clinical Probiotic...,3.b.2 - Total net official development assista...,3.a.1 - Age-standardized prevalence of current...,3.4.1 - Mortality rate attributed to cardiovas...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,3902
2085,Open Opp,Advancing Research on Alzheimer's Disease (AD)...,3.b.2 - Total net official development assista...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,3892
2352,Open Opp,Clinical Trials to Test the Effectiveness of T...,3.4.2 - Suicide mortality rate,3.8.1 - Coverage of essential health services ...,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,3841
2097,Open Opp,"Ethical, Legal, and Social Implications (ELSI)...",3.4.1 - Mortality rate attributed to cardiovas...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,3825


In [21]:
#Obtaining infomation on the maximum, median' and minimum number of words in a comment
print ('Min:', train['Word Count'].min())
print ('Median:', train['Word Count'].median())
print ('Max:', train['Word Count'].max())


Min: 2
Median: 235.0
Max: 4311


In [31]:
# Machine learners like things in X's and Y's!
X = train['Text']
Y = targets[list(submission.columns.values)[1:]].values #converting to an array each of vectors for each comment

In [38]:
# Always check, check, check!
print('X_train shape:', X_train.shape)
print('X_val shape:', X_val.shape)
print('X_test shape:', X_test.shape)
print('Y_train shape:', Y_train.shape)
print('Y_val shape:', Y_val.shape)
print('Y_test shape:', Y_test.shape)

X_train shape: (1916, 300)
X_val shape: (480, 300)
X_test shape: (599, 300)
Y_train shape: (1916, 27)
Y_val shape: (480, 27)
Y_test shape: (599, 27)


# Hyper parameters

In [63]:
# NN batch size
NN_batch_size = 32

# Embedding dimension
embedding_dims = 300

# Number of NN epochs
NN_epochs = 10

In [64]:
K.clear_session()

In [65]:
# Define the model in Keras
model = Sequential()

model.add(Embedding(top_words,
                    embedding_dims,weights=[embedding_matrix],
                    input_length=text_length))

# Dropout
model.add(Dropout(0.5))

# 1-D convolutional layer
model.add(Conv1D(128, 5, padding='valid',activation='relu',strides=1))
model.add(BatchNormalization())

In [66]:
# 1-D convolutional layer
model.add(Conv1D(128, 5, padding='valid',activation='relu',strides=1))
model.add(BatchNormalization())

In [67]:
# This is how we add an LSTM cell. (We can stack multiple cells on top of each other by repeating this line)
model.add(LSTM(52))
model.add(LSTM(52))
model.add(LSTM(52))
model.add(BatchNormalization())
model.add(BatchNormalization())
model.add(BatchNormalization())

TypeError: while_loop() got an unexpected keyword argument 'maximum_iterations'

In [ ]:
# Output layer
model.add(Dense(27, activation = 'sigmoid'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# Check the number of parameters. Always a good idea to take a look at the model from Keras' point of view
model.summary()

In [ ]:
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=10, batch_size=32, verbose=1, shuffle=True)

In [ ]:
# tokenising based on the set out hyper parameters
tokenizer = Tokenizer(top_words)
tokenizer.fit_on_texts(test['Text'].values)

In [ ]:
X_test = test['Text']

In [ ]:
X_sequences_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_test_encoded = pad_sequences(X_sequences_test, text_length)

In [ ]:
y_pred = model.predict([X_test_encoded], batch_size=1024, verbose=1)


In [11]:
bdlstm.head()

,ID,3.1.1,3.1.2,3.2.1,3.2.2,3.3.1,3.3.2,3.3.3,3.3.4,3.3.5,...,3.8.2,3.9.1,3.9.2,3.9.3,3.a.1,3.b.1,3.b.2,3.b.3,3.c.1,3.d.1
0,11437,0.013047,0.006642,0.022350,0.005527,0.121689,0.031064,0.030639,0.014834,0.016384,...,0.009754,0.017295,0.001979,0.010038,0.024244,0.007447,0.549855,0.092907,0.073010,0.046594
1,11474,0.016889,0.026504,0.032181,0.007069,0.063520,0.014994,0.038944,0.004776,0.012255,...,0.052325,0.014660,0.011324,0.017182,0.009629,0.040573,0.525417,0.245046,0.032271,0.123030
2,11475,0.005568,0.005376,0.013778,0.005487,0.010697,0.007539,0.037127,0.007524,0.003784,...,0.023279,0.005779,0.014921,0.004596,0.071478,0.002178,0.098173,0.027276,0.027123,0.069386
3,11476,0.001673,0.000441,0.004871,0.005003,0.051486,0.009920,0.048845,0.042770,0.022051,...,0.001623,0.005418,0.000257,0.003435,0.003415,0.002563,0.616745,0.018472,0.016124,0.012099
4,11486,0.004098,0.004380,0.025361,0.006174,0.072843,0.005124,0.074963,0.009666,0.017083,...,0.013466,0.217359,0.113610,0.112745,0.011400,0.004631,0.386027,0.015429,0.059828,0.045508


In [17]:
bdlstm = pd.read_csv('submission_bdLstm.csv') # PL score 0.9829
cov2d = pd.read_csv('submission_2d.csv') # 0.9811
gru = pd.read_csv('submission_gru.csv') # 0.9788
#lgb = pd.read_csv('../input/lightgbm-with-select-k-best-on-tfidf/lgb_submission.csv') # 0.9785


# The value of an ensemble is (a) the individual scores of the models and
# (b) their correlation with one another. We want multiple individually high
# scoring models that all have low correlations. Based on this analysis, it
# looks like these kernels have relatively low correlations and will blend to a
# much higher score.
labels = [class_names]
for label in labels:
    print(label)
   # print(np.corrcoef([gru[label].rank(pct=True), cov2d[label].rank(pct=True), bdlstm[label].rank(pct=True)]))

submission = pd.DataFrame()
submission['ID'] = gru['ID']
for label in labels:
    submission[label] = gru[label].rank(pct=True) * 0.7 + cov2d[label].rank(pct=True) * 0.15 + bdlstm[label].rank(pct=True) * 0.15 

submission.to_csv('submission_averaged1.csv', index=False)

['3.1.1', '3.1.2', '3.2.1', '3.2.2', '3.3.1', '3.3.2', '3.3.3', '3.3.4', '3.3.5', '3.4.1', '3.4.2', '3.5.1', '3.5.2', '3.6.1', '3.7.1', '3.7.2', '3.8.1', '3.8.2', '3.9.1', '3.9.2', '3.9.3', '3.a.1', '3.b.1', '3.b.2', '3.b.3', '3.c.1', '3.d.1']


In [ ]:
#sample_submission = pd.read_csv(f'{path}{comp}sample_submission.csv')
sample_sub[class_names] = y_pred
sample_sub.to_csv('LSTM-submission.csv', index=False)

In [18]:
submitted=pd.read_csv('submission_averaged1.csv')

In [19]:
for column in submitted.columns[1:]:
    submitted[column]=submitted[column].apply(lambda x: 1 if (x >=.98 ) else 0)
    

In [20]:
submitted.to_csv('./submission_averaged1_98.csv',index=False)